<a name="top"></a><img src="images/chisel_1024.png" alt="Chisel logo" style="width:480px;" />

# Module 3.1: Generators: Parameters
**Prev: [ChiselTest (was chisel-testers2)](2.6_chiseltest.ipynb)**<br>
**Next: [Generators: Collections](3.2_collections.ipynb)**

## Motivation
对于 Chisel 模块来说，要成为代码生成器，必须有一些东西告诉生成器它应该如何完成工作。在本节中，我们讨论模块参数化、各种方法和 Scala 语言特性。参数传递实现的丰富性与生成电路的丰富性成正比。参数应提供有用的默认值，易于设置，并能防止非法或无意义的值。对于更复杂的系统，如果参数能够在本地被覆盖而不会无意中影响其他模块的使用，将会非常有用。

## Setup

In [ ]:
val path = System.getProperty("user.dir") + "/source/load-ivy.sc"
interp.load.module(ammonite.ops.Path(java.nio.file.FileSystems.getDefault().getPath(path)))

In [ ]:
import chisel3._
import chisel3.util._
import chisel3.tester._
import chisel3.tester.RawTester.test

---

# 参数传递
Chisel 提供了强大的构造用于编写硬件生成器。生成器是一些程序，它们接受一些电路参数并生成电路描述。在本节中，我们将从讨论 Chisel 生成器如何获取它们的参数开始。

<span style="color:blue">**Example: Parameterized Scala Object**</span><br>
每个 Chisel `Module` 都是一个普通的 Scala 类。回想一下，Scala 类可以像这样进行参数化：

In [ ]:
class ParameterizedScalaObject(param1: Int, param2: String) {
  println(s"I have parameters: param1 = $param1 and param2 = $param2")
}
val obj1 = new ParameterizedScalaObject(4,     "Hello")
val obj2 = new ParameterizedScalaObject(4 + 2, "World")

<span style="color:blue">**Example: Parameterized Chisel Object**</span><br>
Chisel 模块可以以相同的方式进行参数化。以下模块具有所有输入和输出的宽度参数。运行代码块将生成 Verilog。尝试更改参数并检查输出是否反映了新参数。

In [ ]:
class ParameterizedWidthAdder(in0Width: Int, in1Width: Int, sumWidth: Int) extends Module {
  require(in0Width >= 0)
  require(in1Width >= 0)
  require(sumWidth >= 0)
  val io = IO(new Bundle {
    val in0 = Input(UInt(in0Width.W))
    val in1 = Input(UInt(in1Width.W))
    val sum = Output(UInt(sumWidth.W))
  })
  // a +& b includes the carry, a + b does not
  io.sum := io.in0 +& io.in1
}

println(getVerilog(new ParameterizedWidthAdder(1, 4, 6)))

上面的代码块包含一些 `require(...)` 语句。这些是预编译断言，当你的生成器仅在某些参数化条件下工作或某些参数化条件相互排斥或无意义时，它们非常有用。上面的代码块检查宽度是否为非负数。

有一个单独的结构用于仿真时的断言，称为 `assert(...)`。

## 使用参数化模块进行排序
以下代码块是一个类似于模块 2.3 中 `Sort4` 的参数化排序。与之前具有参数化宽度 IO 的加法器示例不同，此示例具有固定 IO。参数控制在模块内部生成的硬件。
![Sort4](images/Sorter4.png)
<span style="color:blue">**Example: Parameterized 4-Input Sort**</span><br>
与 2.3 不同，此实现参数化为降序或升序排序。

In [ ]:
/** Sort4 sorts its 4 inputs to its 4 outputs */
class Sort4(ascending: Boolean) extends Module {
  val io = IO(new Bundle {
    val in0 = Input(UInt(16.W))
    val in1 = Input(UInt(16.W))
    val in2 = Input(UInt(16.W))
    val in3 = Input(UInt(16.W))
    val out0 = Output(UInt(16.W))
    val out1 = Output(UInt(16.W))
    val out2 = Output(UInt(16.W))
    val out3 = Output(UInt(16.W))
  })
    
  // this comparison funtion decides < or > based on the module's parameterization
  def comp(l: UInt, r: UInt): Bool = {
      if (ascending) {
        l < r
      } else {
        l > r
    }
  }

  val row10 = Wire(UInt(16.W))
  val row11 = Wire(UInt(16.W))
  val row12 = Wire(UInt(16.W))
  val row13 = Wire(UInt(16.W))

  when(comp(io.in0, io.in1)) {
    row10 := io.in0            // preserve first two elements
    row11 := io.in1
  }.otherwise {
    row10 := io.in1            // swap first two elements
    row11 := io.in0
  }

  when(comp(io.in2, io.in3)) {
    row12 := io.in2            // preserve last two elements
    row13 := io.in3
  }.otherwise {
    row12 := io.in3            // swap last two elements
    row13 := io.in2
  }

  val row21 = Wire(UInt(16.W))
  val row22 = Wire(UInt(16.W))

  when(comp(row11, row12)) {
    row21 := row11            // preserve middle 2 elements
    row22 := row12
  }.otherwise {
    row21 := row12            // swap middle two elements
    row22 := row11
  }

  val row20 = Wire(UInt(16.W))
  val row23 = Wire(UInt(16.W))
  when(comp(row10, row13)) {
    row20 := row10            // preserve the first and the forth elements
    row23 := row13
  }.otherwise {
    row20 := row13            // swap the first and the forth elements
    row23 := row10
  }

  when(comp(row20, row21)) {
    io.out0 := row20            // preserve first two elements
    io.out1 := row21
  }.otherwise {
    io.out0 := row21            // swap first two elements
    io.out1 := row20
  }

  when(comp(row22, row23)) {
    io.out2 := row22            // preserve first two elements
    io.out3 := row23
  }.otherwise {
    io.out2 := row23            // swap first two elements
    io.out3 := row22
  }
}



// Here are the testers
test(new Sort4(true)) { c => 
  c.io.in0.poke(3.U)
  c.io.in1.poke(6.U)
  c.io.in2.poke(9.U)
  c.io.in3.poke(12.U)
  c.io.out0.expect(3.U)
  c.io.out1.expect(6.U)
  c.io.out2.expect(9.U)
  c.io.out3.expect(12.U)

  c.io.in0.poke(13.U)
  c.io.in1.poke(4.U)
  c.io.in2.poke(6.U)
  c.io.in3.poke(1.U)
  c.io.out0.expect(1.U)
  c.io.out1.expect(4.U)
  c.io.out2.expect(6.U)
  c.io.out3.expect(13.U)

  c.io.in0.poke(13.U)
  c.io.in1.poke(6.U)
  c.io.in2.poke(4.U)
  c.io.in3.poke(1.U)
  c.io.out0.expect(1.U)
  c.io.out1.expect(4.U)
  c.io.out2.expect(6.U)
  c.io.out3.expect(13.U)
}
test(new Sort4(false)) { c =>
  c.io.in0.poke(3.U)
  c.io.in1.poke(6.U)
  c.io.in2.poke(9.U)
  c.io.in3.poke(12.U)
  c.io.out0.expect(12.U)
  c.io.out1.expect(9.U)
  c.io.out2.expect(6.U)
  c.io.out3.expect(3.U)

  c.io.in0.poke(13.U)
  c.io.in1.poke(4.U)
  c.io.in2.poke(6.U)
  c.io.in3.poke(1.U)
  c.io.out0.expect(13.U)
  c.io.out1.expect(6.U)
  c.io.out2.expect(4.U)
  c.io.out3.expect(1.U)

  c.io.in0.poke(1.U)
  c.io.in1.poke(6.U)
  c.io.in2.poke(4.U)
  c.io.in3.poke(13.U)
  c.io.out0.expect(13.U)
  c.io.out1.expect(6.U)
  c.io.out2.expect(4.U)
  c.io.out3.expect(1.U)
}
println("SUCCESS!!") // Scala Code: if we get here, our tests passed!

---
# Option and Default Arguments

有时一个函数有时返回一个值，有时则不返回值。Scala 有一种机制可以在类型系统中对这种情况进行编码，而不是在无法返回值时抛出错误。

<span style="color:blue">**示例：错误的 Map 索引调用**</span><br>
在下面的示例中，我们有一个包含多个键/值对的 map。如果我们尝试访问缺失的键/值对，则会遇到运行时错误：

In [ ]:
val map = Map("a" -> 1)
val a = map("a")
println(a)
val b = map("b")
println(b)

<span style="color:blue">**示例：获取不确定的索引**</span><br>
然而，`Map` 提供了另一种通过 **get** 方法访问键值的方式。使用此方法返回抽象类 `Option` 的值。`Option` 有两个子类，`Some` 和 `None`。

In [ ]:
val map = Map("a" -> 1)
val a = map.get("a")
println(a)
val b = map.get("b")
println(b)

如你将在后面的章节中看到的，`Option` 非常重要，因为它允许用户使用 match 语句来检查 Scala 类型和值。

<span style="color:blue">**示例：获取或其他！**</span><br>
像 `Map` 一样，`Option` 也有一个 `get` 方法，如果在 `None` 上调用它会出错。在这些情况下，我们可以使用 **`getOrElse`** 提供一个默认值。

In [ ]:
val some = Some(1)
val none = None
println(some.get)          // Returns 1
// println(none.get)       // Errors!
println(some.getOrElse(2)) // Returns 1
println(none.getOrElse(2)) // Returns 2

## Options for Parameters with Defaults

当对象或函数有很多参数时，始终完全指定它们可能既繁琐又容易出错。在模块 1 中，你已经了解了命名参数和参数默认值。有时，参数没有一个好的默认值。在这些情况下，可以使用 `Option` 并将其默认值设置为 `None`。

<span style="color:blue">**示例：可选复位**</span><br>
以下显示了一个将输入延迟一个时钟周期的代码块。如果 `resetValue = None`（这是默认值），则寄存器将没有复位值，并被初始化为垃圾。这避免了使用超出正常范围的值来表示“无”的常见但丑陋的情况，例如使用 -1 作为复位值来表示此寄存器不复位。

In [ ]:
class DelayBy1(resetValue: Option[UInt] = None) extends Module {
    val io = IO(new Bundle {
        val in  = Input( UInt(16.W))
        val out = Output(UInt(16.W))
    })
    val reg = if (resetValue.isDefined) { // resetValue = Some(number)
        RegInit(resetValue.get)
    } else { //resetValue = None
        Reg(UInt())
    }
    reg := io.in
    io.out := reg
}

println(getVerilog(new DelayBy1))
println(getVerilog(new DelayBy1(Some(3.U))))

---
# Match/Case Statements
Scala 的 *matching* 概念在 Chisel 中被广泛使用，需要成为任何 Chisel 程序员的基本理解的一部分。Scala 提供了 match 操作符，它支持：
- 简单的替代测试，有点类似于 C 的 *switch* 语句
- 对值的临时组合进行更复杂的测试
- 根据变量类型未知或不确定时的类型采取行动，例如当
  - 变量来自异构列表 ```val mixedList = List(1, "string", false)```
  - 或者变量被认为是超类的成员但不是特定子类的成员
- 提取用 *正则表达式* 指定的字符串子串

<span style="color:blue">**示例：值匹配**</span><br>
以下示例中，根据我们 **match** 的变量的 **值**，执行不同的 **case** 语句：

In [ ]:
// y is an integer variable defined somewhere else in the code
val y = 7
/// ...
val x = y match {
  case 0 => "zero" // One common syntax, preferred if fits in one line
  case 1 =>        // Another common syntax, preferred if does not fit in one line.
      "one"        // Note the code block continues until the next case
  case 2 => {      // Another syntax, but curly braces are not required
      "two"
  }
  case _ => "many" // _ is a wildcard that matches all values
}
println("y is " + x)

match 操作符检查可能的值，并为每个 case 返回一个字符串。有几点需要注意：
- 跟随 `=>` 操作符的每个代码块会继续执行，直到到达 match 的结束括号或下一个 case 语句。
- match 按照 case 语句的顺序进行搜索，一旦匹配到一个 case 语句，不再检查其他 case 语句。
- 使用下划线作为通配符，处理未找到的任何值。

<span style="color:blue">**示例：多值匹配**</span><br>
此外，可以同时匹配多个变量。以下是使用 match 语句和值元组实现的真值表的一个简单示例：

In [ ]:
def animalType(biggerThanBreadBox: Boolean, meanAsCanBe: Boolean): String = {
  (biggerThanBreadBox, meanAsCanBe) match {
    case (true, true) => "wolverine"
    case (true, false) => "elephant"
    case (false, true) => "shrew"
    case (false, false) => "puppy"
  }
}
println(animalType(true, true))

<span style="color:blue">**示例：类型匹配**</span><br>
Scala 是一种强类型语言，因此在运行时所有对象的类型都是已知的。我们可以使用 **match 语句** 利用这些类型信息来控制流程：

In [ ]:
val sequence = Seq("a", 1, 0.0)
sequence.foreach { x =>
  x match {
    case s: String => println(s"$x is a String")
    case s: Int    => println(s"$x is an Int")
    case s: Double => println(s"$x is a Double")
    case _ => println(s"$x is an unknown type!")
  }
}

<span style="color:blue">**示例：多类型匹配**</span><br>
如果要匹配一个值是否具有多种类型之一，请使用以下语法。*注意，在匹配时**必须**使用 `_`。*

In [ ]:
val sequence = Seq("a", 1, 0.0)
sequence.foreach { x =>
  x match {
    case _: Int | _: Double => println(s"$x is a number!")
    case _ => println(s"$x is an unknown type!")
  }
}

<span style="color:blue">**示例：类型匹配和擦除**</span><br>
类型匹配有一些限制。由于 Scala 运行在 JVM 上，而 JVM 不维护多态类型，因此你不能在运行时匹配它们（因为它们都被擦除了）。请注意，以下示例总是匹配第一个 case 语句，因为 `[String]`、`[Int]` 和 `[Double]` 多态类型被擦除，case 语句**实际上**只是匹配一个 `Seq`。

In [ ]:
val sequence = Seq(Seq("a"), Seq(1), Seq(0.0))
sequence.foreach { x =>
  x match {
    case s: Seq[String] => println(s"$x is a String")
    case s: Seq[Int]    => println(s"$x is an Int")
    case s: Seq[Double] => println(s"$x is a Double")
  }
}

请注意，如果你实现像上面示例那样的代码，Scala 编译器通常会发出警告。

（译者添加）要正确匹配类型，可以通过检查序列中元素的具体类型来实现。以下是改进的代码：

In [ ]:
val sequence = Seq(Seq("a"), Seq(1), Seq(0.0))
sequence.foreach { x =>
  x.headOption match {
    case Some(_: String) => println(s"$x is a Seq of String")
    case Some(_: Int)    => println(s"$x is a Seq of Int")
    case Some(_: Double) => println(s"$x is a Seq of Double")
    case _               => println(s"$x is an unknown type!")
  }
}

在这个改进的代码中：

- headOption 方法：获取序列中的第一个元素并返回一个 Option，如果序列为空，则返回 None。
- 显式类型检查：通过模式匹配检查 headOption 返回的 Option 的内容类型。这避免了类型擦除的问题，因为我们检查的是具体元素的类型，而不是泛型类型。

<span style="color:blue">**示例：可选复位匹配**</span><br>
下面的代码块显示了使用 match 结构而不是 `if/else` 的相同 `DelayBy1` 模块。

In [ ]:
class DelayBy1(resetValue: Option[UInt] = None) extends Module {
  val io = IO(new Bundle {
    val in  = Input( UInt(16.W))
    val out = Output(UInt(16.W))
  })
  val reg = resetValue match {
    case Some(r) => RegInit(r)
    case None    => Reg(UInt())
  }
  reg := io.in
  io.out := reg
}

println(getVerilog(new DelayBy1))
println(getVerilog(new DelayBy1(Some(3.U))))

---
# IOs with Optional Fields

有时我们希望 IO 是可选包含或排除的。也许有一些内部状态在调试时查看会很好，但当生成器在系统中使用时你希望隐藏它。也许你的生成器有一些输入不需要在每种情况下都连接，因为有一个合理的默认值。

<span style="color:blue">**示例：带有 Option 的可选 IO**</span><br>
可选的 bundle 字段是一种实现此功能的方法。在下面的示例中，我们展示了一个一位加法器，它可以选择性地接收一个进位。如果包含进位，`io.carryIn` 将具有 `Some[UInt]` 类型，并包含在 IO bundle 中。如果不包含进位，`io.carryIn` 将具有 `None` 类型，并将被排除在 IO bundle 之外。

In [ ]:
class HalfFullAdder(val hasCarry: Boolean) extends Module {
  val io = IO(new Bundle {
    val a = Input(UInt(1.W))
    val b = Input(UInt(1.W))
    val carryIn = if (hasCarry) Some(Input(UInt(1.W))) else None
    val s = Output(UInt(1.W))
    val carryOut = Output(UInt(1.W))
  })
  val sum = io.a +& io.b +& io.carryIn.getOrElse(0.U)
  io.s := sum(0)
  io.carryOut := sum(1)
}

test(new HalfFullAdder(false)) { c =>
  require(!c.hasCarry, "DUT must be half adder")
  // 0 + 0 = 0
  c.io.a.poke(0.U)
  c.io.b.poke(0.U)
  c.io.s.expect(0.U)
  c.io.carryOut.expect(0.U)
  // 0 + 1 = 1
  c.io.b.poke(1.U)
  c.io.s.expect(1.U)
  c.io.carryOut.expect(0.U)
  // 1 + 1 = 2
  c.io.a.poke(1.U)
  c.io.s.expect(0.U)
  c.io.carryOut.expect(1.U)
  // 1 + 0 = 1
  c.io.b.poke(0.U)
  c.io.s.expect(1.U)
  c.io.carryOut.expect(0.U)
}

test(new HalfFullAdder(true)) { c =>
  require(c.hasCarry, "DUT must be half adder")
  c.io.carryIn.get.poke(0.U)
  // 0 + 0 + 0 = 0
  c.io.a.poke(0.U)
  c.io.b.poke(0.U)
  c.io.s.expect(0.U)
  c.io.carryOut.expect(0.U)
  // 0 + 0 + 1 = 1
  c.io.b.poke(1.U)
  c.io.s.expect(1.U)
  c.io.carryOut.expect(0.U)
  // 0 + 1 + 1 = 2
  c.io.a.poke(1.U)
  c.io.s.expect(0.U)
  c.io.carryOut.expect(1.U)
  // 0 + 1 + 0 = 1
  c.io.b.poke(0.U)
  c.io.s.expect(1.U)
  c.io.carryOut.expect(0.U)

  c.io.carryIn.get.poke(1.U)
  // 1 + 0 + 0 = 1
  c.io.a.poke(0.U)
  c.io.b.poke(0.U)
  c.io.s.expect(1.U)
  c.io.carryOut.expect(0.U)
  // 1 + 0 + 1 = 2
  c.io.b.poke(1.U)
  c.io.s.expect(0.U)
  c.io.carryOut.expect(1.U)
  // 1 + 1 + 1 = 3
  c.io.a.poke(1.U)
  c.io.s.expect(1.U)
  c.io.carryOut.expect(1.U)
  // 1 + 1 + 0 = 2
  c.io.b.poke(0.U)
  c.io.s.expect(0.U)
  c.io.carryOut.expect(1.U)
}

println("SUCCESS!!") // Scala Code: if we get here, our tests passed!

<span style="color:blue">**示例：带有零宽度导线的可选 IO**</span><br>
实现与 `Option` 类似功能的另一种方法是使用零宽度导线。Chisel 类型允许具有零宽度。宽度为零的 IO 会在生成的 Verilog 中被修剪掉，任何尝试使用零宽度导线值的操作都会得到一个常量零。如果零是一个合理的默认值，零宽度导线非常好，因为它们不需要对 `Option` 进行匹配或调用 `getOrElse`。

In [ ]:
class HalfFullAdder(val hasCarry: Boolean) extends Module {
  val io = IO(new Bundle {
    val a = Input(UInt(1.W))
    val b = Input(UInt(1.W))
    val carryIn = Input(if (hasCarry) UInt(1.W) else UInt(0.W))
    val s = Output(UInt(1.W))
    val carryOut = Output(UInt(1.W))
  })
  val sum = io.a +& io.b +& io.carryIn
  io.s := sum(0)
  io.carryOut := sum(1)
}
println("Half Adder:")
println(getVerilog(new HalfFullAdder(false)))
println("\n\nFull Adder:")
println(getVerilog(new HalfFullAdder(true)))

---
# Implicits
编程时常常需要大量的样板代码。为了解决这种情况，Scala 引入了 **implicits** 的概念，它允许编译器为你做一些语法糖处理。由于很多事情都在幕后发生，implicits 看起来非常神奇。本节通过一些基本示例来解释它们是什么以及它们常用的地方。

## Implicit Arguments
有时，你的代码需要从一系列函数调用的深处访问某种顶层变量。与其手动通过每个函数调用传递此变量，不如使用 implicit 参数来为你完成这项工作。

<span style="color:blue">**示例：隐式 Cats**</span><br>
在下面的示例中，我们可以隐式或显式地传递猫的数量。

In [ ]:
object CatDog {
  implicit val numberOfCats: Int = 3
  //implicit val numberOfDogs: Int = 5

  def tooManyCats(nDogs: Int)(implicit nCats: Int): Boolean = nCats > nDogs
    
  val imp = tooManyCats(2)    // Argument passed implicitly!
  val exp = tooManyCats(2)(1) // Argument passed explicitly!
}
CatDog.imp
CatDog.exp

发生了什么？首先，我们定义了一个隐式值 **numberOfCats**。在给定的作用域中，**只能有一个给定类型的隐式值**。然后，我们定义了一个函数，它接受两个参数列表；第一个是任何显式参数，第二个是任何隐式参数。当我们调用 **tooManyCats** 时，我们可以省略第二个隐式参数列表（让编译器为我们找到它），也可以显式地提供一个参数（可以与隐式值不同）。

隐式参数可能*失败*的情况如下：
- 在一个作用域中定义了两个或多个给定类型的隐式值
- 编译器找不到函数调用所需的隐式值

<span style="color:blue">**示例：隐式日志记录**</span><br>
下一个代码块展示了如何使用隐式参数在 Chisel 生成器中实现日志记录。

***注意：在 Scala 中有更好的日志记录方法！***

In [ ]:
sealed trait Verbosity
implicit case object Silent extends Verbosity
case object Verbose extends Verbosity
// Silent和Verbose是Verbosity trait的两种实例，分别表示静默和详细日志记录。

class ParameterizedWidthAdder(in0Width: Int, in1Width: Int, sumWidth: Int)(implicit verbosity: Verbosity)
extends Module {
  def log(msg: => String): Unit = verbosity match {
    case Silent =>
    case Verbose => println(msg)
  }
  require(in0Width >= 0)
  log(s"in0Width of $in0Width OK")
  require(in1Width >= 0)
  log(s"in1Width of $in1Width OK")
  require(sumWidth >= 0)
  log(s"sumWidth of $sumWidth OK")
  val io = IO(new Bundle {
    val in0 = Input(UInt(in0Width.W))
    val in1 = Input(UInt(in1Width.W))
    val sum = Output(UInt(sumWidth.W))
  })
  log("Made IO")
  io.sum := io.in0 + io.in1
  log("Assigned output")
}

println(getVerilog(new ParameterizedWidthAdder(1, 4, 5)))
println(getVerilog(new ParameterizedWidthAdder(1, 4, 5)(Verbose)))

（译者添加）对于上面 log 函数的解释。

参数`msg: => String`

- `msg: => String`是一种“按需计算”参数（call-by-name parameter）。与普通的`msg: String`不同，这种参数在传递时不会立即计算其值，而是在使用时才计算。
- 这意味着当`log`函数被调用时，只有在`verbosity`是`Verbose`的情况下，`msg`才会被计算并传递给`println`。如果`verbosity`是`Silent`，`msg`则不会被计算。

为什么使用这种语法？  
由于`msg`是按需计算的参数，如果`verbosity`是`Silent`，那么`msg`的计算会被跳过。这在某些情况下可以避免不必要的计算，从而提高程序的效率。只有在确定需要打印日志时，才计算日志消息的内容。这对于构建复杂的日志消息（例如包含大量变量的字符串拼接）非常有用，可以减少不必要的性能开销。

考虑以下示例，演示按需计算参数的效果：

```scala
def expensiveComputation(): String = {
  println("Expensive computation is performed")
  "Result of computation"
}

def log(msg: => String): Unit = verbosity match {
  case Silent =>
  case Verbose => println(msg)
}

implicit val verbosity: Verbosity = Silent
log(expensiveComputation())  // This will not print "Expensive computation is performed"
```

在上面的示例中，由于`verbosity`是`Silent`，`expensiveComputation`函数不会被调用，因为`msg`参数没有被使用。这就避免了不必要的计算。

## Implicit Conversions
和隐式参数一样，隐式函数（也称为**隐式转换**）用于减少样板代码。更具体地说，它们用于自动将一个 Scala 对象转换为另一个。

<span style="color:blue">**示例：隐式转换**</span><br>
在下面的示例中，我们有两个类，`Animal` 和 `Human`。`Animal` 有一个 `species` 字段，但 `Human` 没有。然而，通过实现隐式转换，我们可以在 `Human` 上调用 `species`。

In [ ]:
class Animal(val name: String, val species: String)
class Human(val name: String)
implicit def human2animal(h: Human): Animal = new Animal(h.name, "Homo sapiens")
val me = new Human("Adam")
println(me.species)

通常，隐式转换会使你的代码变得混乱，因此我们建议你将其作为最后的手段。首先尝试继承、特征或方法重载。

---
# Generator Example
以下示例展示了一个用于 1 位输入 Mealy 机器的生成器。它的测试基于 [维基百科](https://en.wikipedia.org/wiki/Mealy_machine#/media/File:Mealy.png) 的示例。阅读代码并尝试理解其运行过程。

<span style="color:blue">**示例：Mealy 机器**</span><br>
尝试对 Mealy 机器生成器进行自己的参数化，并在下面的代码块中编写自己的测试。

In [ ]:
// Mealy machine has
case class BinaryMealyParams(
  // number of states
  nStates: Int,
  // initial state
  s0: Int,
  // function describing state transition
  stateTransition: (Int, Boolean) => Int,
  // function describing output
  output: (Int, Boolean) => Int
) {
  require(nStates >= 0)
  require(s0 < nStates && s0 >= 0)
}

class BinaryMealy(val mp: BinaryMealyParams) extends Module {
  val io = IO(new Bundle {
    val in = Input(Bool())
    val out = Output(UInt())
  })

  val state = RegInit(UInt(), mp.s0.U)

  // output zero if no states
  io.out := 0.U
  for (i <- 0 until mp.nStates) {
    when (state === i.U) {
      when (io.in) {
        state  := mp.stateTransition(i, true).U
        io.out := mp.output(i, true).U
      }.otherwise {
        state  := mp.stateTransition(i, false).U
        io.out := mp.output(i, false).U
      }
    }
  }
}

// example from https://en.wikipedia.org/wiki/Mealy_machine
val nStates = 3
val s0 = 2
def stateTransition(state: Int, in: Boolean): Int = {
  if (in) {
    1
  } else {
    0
  }
}
def output(state: Int, in: Boolean): Int = {
  if (state == 2) {
    return 0
  }
  if ((state == 1 && !in) || (state == 0 && in)) {
    return 1
  } else {
    return 0
  }
}

val testParams = BinaryMealyParams(nStates, s0, stateTransition, output)

test(new BinaryMealy(testParams)) { c =>
  c.io.in.poke(false.B)
  c.io.out.expect(0.U)
  c.clock.step(1)
  c.io.in.poke(false.B)
  c.io.out.expect(0.U)
  c.clock.step(1)
  c.io.in.poke(false.B)
  c.io.out.expect(0.U)
  c.clock.step(1)
  c.io.in.poke(true.B)
  c.io.out.expect(1.U)
  c.clock.step(1)
  c.io.in.poke(true.B)
  c.io.out.expect(0.U)
  c.clock.step(1)
  c.io.in.poke(false.B)
  c.io.out.expect(1.U)
  c.clock.step(1)
  c.io.in.poke(true.B)
  c.io.out.expect(1.U)
  c.clock.step(1)
  c.io.in.poke(false.B)
  c.io.out.expect(1.U)
  c.clock.step(1)
  c.io.in.poke(true.B)
  c.io.out.expect(1.U)
}

println("SUCCESS!!") // Scala Code: if we get here, our tests passed!

---
# You're done!

[Return to the top.](#top)